# IMPORTS

## Libraries

In [1]:
import pandas as pd
import numpy as np
import math
from google.oauth2 import service_account
from googleapiclient.discovery import build

pd.set_option('display.max_columns', 200)

# Read to Google Sheets

In [16]:
# Documentation: https://developers.google.com/sheets/api/quickstart/python
SERVICE_ACCOUNT_FILE = 'D:/01-DataScience/04-Projetos/00-Git/Youtube-Video-Recommendations/Credentials/keys.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

credentials = None
credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)


# The ID of spreadsheet.
SAMPLE_SPREADSHEET_ID = '1uCur7jOXuLnwuwfWgoBL8mvDDvchuLf-o0X-AnOxS7s'

service = build('sheets', 'v4', credentials=credentials)


# Call the Sheets API
sheet = service.spreadsheets()

#Read Values
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range="YoutubeScrapeSample!A1:R").execute()

values = result.get('values', [])

In [46]:
dfGoogleSheets = pd.DataFrame(values[1:], columns=values[0])


In [59]:
dfGoogleSheets['UploadDate'] = pd.to_datetime(dfGoogleSheets['UploadDate'])
dfGoogleSheets['WatchList'] = dfGoogleSheets['WatchList'].replace('', np.nan)
dfGoogleSheets[['DaysSincePublication', 'Duration', 'ViewCount', 'LikeCount', 'DislikeCount']] = dfGoogleSheets[['DaysSincePublication', 'Duration', 'ViewCount', 'LikeCount', 'DislikeCount']].astype(int)
dfGoogleSheets['AverageRating'] = dfGoogleSheets['AverageRating'].astype(float)
dfGoogleSheets[dfGoogleSheets.select_dtypes(include=['object']).columns] = dfGoogleSheets.select_dtypes(include=['object']).astype('category')

In [60]:
dfGoogleSheets.dtypes

WatchList                     category
Id                            category
Uploader                      category
UploadDate              datetime64[ns]
Title                         category
Description                   category
Categories                    category
Tags                          category
Duration                         int32
ViewCount                        int32
LikeCount                        int32
DislikeCount                     int32
AverageRating                  float64
Query                         category
DaysSincePublication             int32
ThumbnailUrl                  category
ThumbnailWidth                category
ThumbnailHeight               category
dtype: object

In [70]:
dfTrain = dfGoogleSheets[dfGoogleSheets['WatchList'].notnull()].reset_index(drop=True)
dfTest = dfGoogleSheets[dfGoogleSheets['WatchList'].isnull()].reset_index(drop=True)

In [71]:
# Train Dataset
print('\n\tTrain Dataset')
print(f'Number of Rows: {dfTrain.shape[0]}')
print(f'Number of Columns: {dfTrain.shape[1]}')

# Test Dataset
print('\n\n\tTest Dataset')
print(f'Number of Rows: {dfTest.shape[0]}')
print(f'Number of Columns: {dfTest.shape[1]}')


	Train Dataset
Number of Rows: 800
Number of Columns: 18


	Test Dataset
Number of Rows: 3582
Number of Columns: 18


# Convert to .feather

In [72]:
dfTrain.to_feather('../Data/FeatherData/dfTrainGS.feather')
dfTest.to_feather('../Data/FeatherData/dfTestGS.feather')